In [1]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import os

In [2]:
import re

def refine_script(script):
    noise_patterns = [
        "안녕하세요?", "예.", "네.", "알겠습니다.", "수고하십니다.", "감사합니다."
    ]
    
    lines = script.split('\n')
    refined_lines = []
    
    for line in lines:
        line = line.strip()
        if line.startswith("손님:"):
            # "손님:" 태그 제거
            content = line.replace("손님:", "").strip()
            
            # 문장 안에 포함된 노이즈 패턴들을 하나씩 찾아서 ""(빈칸)으로 변경
            for pattern in noise_patterns:
                content = content.replace(pattern, "")
            
            # 양끝 공백 정리
            content = content.strip()
            
            # 만약 노이즈를 다 지웠더니 남은 내용이 너무 짧으면(5자 이하) 빈칸 처리
            if len(content) <= 4:
                continue
            
            refined_lines.append(content)
            
    # 리스트를 공백 하나를 사이에 두고 합침
    result = " ".join(refined_lines)
    
    # 연속된 공백(빈 문자열 때문에 생긴 것들)을 하나로 줄임
    return re.sub(r'\s+', ' ', result).strip()

In [ ]:
import pandas as pd 
import os

# 파일 경로 설정
testset = "trainset.csv"
results_list = []

df = pd.read_csv(testset)
data_list = df.to_dict('records')
    
for item in data_list:
    # 데이터 추출
    script = item.get('consulting_content', "")
    truth = item.get('personality', "")
    script = refine_script(script)
    
    # 결과 저장
    results_list.append({
        "id": item.get('source_id'),
        "script": script,
        "truth": truth,
    })
    
df_new = pd.json_normalize(results_list)
output_file = "train.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: train.csv


---

In [2]:
SYSTEM_PROMPT = """
카드사 콜센터 고객 발화 텍스트를 보고 고객 성향을 1개로 분류하세요

### 출력 규칙
1. 제시된 성향 키워드 중 가장 적절한 한가지만 선택
2. 설명이나 판단 근거는 절대 출력하지말고 오직 하나의 키워드만 출력한다
3. 여러 개의 성향을 가질 경우 S1 > S2 > S3 > N3 > N2 > N1 의 순서로 우선 순위를 가진다

### 성향 키워드 목록
- N1: 일반형. 큰 특징이 없고 바로 문의사항을 말함
- N2: 수다형. 사적인 이야기나 본인 상황을 길게 설명함
- N3: 신중형. 신중하고 의심을 보임
- S1: 급한성격형. 빠른 처리를 선호함
- S2: 이해부족형. 설명을 잘 이해하지 못하여 반복적으로 확인함
- S3: 불만형. 분노, 짜증을 드러냄
"""

In [4]:
import time
import openai

def get_personality_gpt(script):
    start_time = time.perf_counter()
    ttft = 0
    generated_tokens = 0
    content = ""
    first_token_received = False
    
    try:
        response = openai.chat.completions.create(
           model="gpt-4.1-mini",
           messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": script}
            ],
            temperature=0,
            stream=True,                              # 토큰 단위로 수신
            stream_options={"include_usage": True}    # 마지막 chunk에 토큰 정보 포함
        )
        
        for chunk in response:
            # 첫번째 토큰이 들어오는 시점 확인 (ttft)
            if not first_token_received and chunk.choices and chunk.choices[0].delta.content:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            # 내용 누적
            if chunk.choices and chunk.choices[0].delta.content:
                content += chunk.choices[0].delta.content
            
            # 토큰 사용량 확인
            if chunk.usage is not None:
                generated_tokens = chunk.usage.completion_tokens
        
        end_time = time.perf_counter()
        total_duration = end_time - start_time
        
        # tps 계산 (생성된 토큰 수 / 전체 소요 시간)
        tps = generated_tokens / total_duration if total_duration > 0 else 0
        
        metrics = {
                "ttft": round(ttft, 3),
                "tps": round(tps, 2),
                "total_tokens": generated_tokens
            }

        return content, metrics
    except Exception as e:
        return f"호출 중 오류 발생 : {e}", "오류"

In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import time

load_dotenv()

client = OpenAI(
    base_url=os.getenv("RUNPOD_URL"),
    api_key=os.getenv("API_KEY")
)

def get_personality_sllm(script):
    try:
        start_time = time.perf_counter() # 전체 시작 시간
        first_token_time = None
        result = ""
                
        # 스트리밍 호출 활성화
        response = client.chat.completions.create(
            model="ansui/customer-analysis-merged",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": script}
            ],
            temperature=0.0,
            stream=True,                            # 스트리밍 활성화
            max_tokens=10,
            stop=["[|", "[|end|]", "[|user|]", "\n"],
            stream_options={"include_usage": True}  # 토큰 사용량 포함
        )

        total_tokens = 0
        
        for chunk in response:
            # TTFT 계산 (첫 번째 토큰이 들어온 시점)
            if first_token_time is None and chunk.choices and chunk.choices[0].delta.content:
                first_token_time = time.perf_counter()
            
            # 내용 합치기
            if chunk.choices and chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
            
            # 토큰 수
            if chunk.usage:
                total_tokens = chunk.usage.completion_tokens
        
        if "[" in result:
            result = result.split("[")[0].strip()
            
        # 혹시나 포함되어 있을 수 있는 태그 추가 정제
        result = result.replace("[|assistant|]", "").replace("[|end|]", "").strip()

        end_time = time.perf_counter()
        
        # 지표 계산
        ttft = first_token_time - start_time if first_token_time else 0
        total_latency = end_time - start_time
        
        # TPS 계산 (생성된 토큰 수 / 생성에 걸린 시간)
        if total_tokens == 0:
            total_tokens = len(result) / 1.5 
            
        tps = total_tokens / total_latency if total_latency > 0 else 0
        
        metrics = {
            "ttft": round(ttft, 3),
            "tps": round(tps, 2),
            "total_tokens": int(total_tokens)
        }

        return result, metrics

    except Exception as e:
        return {"error": f"오류 발생: {str(e)}"}, {"ttft": 0, "tps": 0, "total_tokens": 0}

In [9]:
import pandas as pd 

# 파일 경로 설정
testset = "test.csv"
results_list = []

df = pd.read_csv(testset)
data_list = df.to_dict('records')
    
for item in data_list:
    # 데이터 추출
    script = item.get('script', "")
    truth = item.get('truth', "")

    print(f"테스트 실행 중: {item.get('id', 'Unknown ID')}")
    res, metrics = get_personality_sllm(script)
    # res, metrics = get_personality_gpt(script)
        
    # 결과 저장
    results_list.append({
        "id": item.get('id'),
        "truth": truth,
        "res": res,
        "metrics": metrics,
    })

테스트 실행 중: 21934
테스트 실행 중: 22715
테스트 실행 중: 25992
테스트 실행 중: 23744
테스트 실행 중: 25401
테스트 실행 중: 27281
테스트 실행 중: 24194
테스트 실행 중: 21917
테스트 실행 중: 24754
테스트 실행 중: 20805
테스트 실행 중: 24252
테스트 실행 중: 25346
테스트 실행 중: 27007
테스트 실행 중: 22524
테스트 실행 중: 25485
테스트 실행 중: 25486
테스트 실행 중: 25246
테스트 실행 중: 25443
테스트 실행 중: 21722
테스트 실행 중: 23847
테스트 실행 중: 24606
테스트 실행 중: 23249
테스트 실행 중: 24435
테스트 실행 중: 26003
테스트 실행 중: 20759
테스트 실행 중: 24415
테스트 실행 중: 23098
테스트 실행 중: 22856
테스트 실행 중: 20742
테스트 실행 중: 25488
테스트 실행 중: 23072
테스트 실행 중: 21499
테스트 실행 중: 20892
테스트 실행 중: 27127
테스트 실행 중: 26930
테스트 실행 중: 27127
테스트 실행 중: 23072
테스트 실행 중: 24950
테스트 실행 중: 200081
테스트 실행 중: 22638
테스트 실행 중: 27322
테스트 실행 중: 23072
테스트 실행 중: 22198
테스트 실행 중: 27262
테스트 실행 중: 21859
테스트 실행 중: 21536
테스트 실행 중: 23561
테스트 실행 중: 200144
테스트 실행 중: 25326
테스트 실행 중: 24712
테스트 실행 중: 22554
테스트 실행 중: 25259
테스트 실행 중: 20670
테스트 실행 중: 24401
테스트 실행 중: 25058
테스트 실행 중: 24119
테스트 실행 중: 21363
테스트 실행 중: 25635
테스트 실행 중: 25088
테스트 실행 중: 22172
테스트 실행 중: 25993
테스트 실행 중: 23989
테스트 실행

In [10]:
import os
import pandas as pd

df_new = pd.json_normalize(results_list)
df_new['model'] = "fine-tuned"

output_file = "results_personality.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: results_personality.csv


---

In [11]:
import pandas as pd
import numpy as np

def analyze_model_performance(file_path):
    try:
        # 데이터 로드
        df = pd.read_csv(file_path, on_bad_lines='skip')
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return None

    # 정확도 판별 (True/False)
    df['is_correct'] = (df['truth'] == df['res'])

    # 1. 모델별 종합 지표 (기존 기능)
    model_summary = df.groupby('model').agg({
        'is_correct': lambda x: x.mean() * 100,
        'metrics.ttft': 'mean',
        'metrics.tps': 'mean',
        'metrics.total_tokens': 'mean'
    })
    model_summary.columns = ['전체 정확도(%)', '평균 TTFT', '평균 TPS', '평균 토큰수']

    # 2. 키워드(truth)별 정확도 분석 (신규 기능)
    # 모델별, 키워드별로 그룹화하여 정확도 계산
    keyword_accuracy = df.groupby(['model', 'truth'])['is_correct'].mean() * 100
    
    # 보기 좋게 피벗 테이블로 변환 (행: 모델, 열: 키워드)
    keyword_pivot = keyword_accuracy.unstack()
    
    order = ['N1', 'N2', 'N3', 'S1', 'S2', 'S3']
    existing_order = [k for k in order if k in keyword_pivot.columns]
    keyword_pivot = keyword_pivot[existing_order]

    return model_summary, keyword_pivot

# --- 실행 및 결과 확인 ---
summary, keywords = analyze_model_performance('results_personality.csv')

print("=== [1] 모델별 종합 성능 ===")
print(summary)

print("\n=== [2] 키워드별 상세 정확도 (%) ===")
print(keywords)

# 파일로 각각 저장
keywords.to_csv('keywords_summary.csv', encoding='utf-8-sig')

=== [1] 모델별 종합 성능 ===
            전체 정확도(%)   평균 TTFT    평균 TPS  평균 토큰수
model                                            
fine-tuned  47.916667  0.736507  5.445833     4.0

=== [2] 키워드별 상세 정확도 (%) ===
truth         N1    N2    N3    S1   S2    S3
model                                        
fine-tuned  65.5  40.5  32.5  72.0  5.0  72.0
